In [3]:
"""
An example reward method
"""

import bittensor as bt

RATE_OF_RECOVERY = 0.2
RATE_OF_DECAY = 0.8

MINIMUM_SCORE = 0

RESPONSE_TIME_WEIGHT = 0.2
PROOF_SIZE_WEIGHT = 0.1

RESPONSE_TIME_THRESHOLD = 40
PROOF_SIZE_THRESHOLD = 30000


import torch
import torch.nn as nn


class Reward(nn.Module):
    def __init__(self):
        super(Reward, self).__init__()
        self.rate_of_decay = RATE_OF_DECAY
        self.rate_of_recovery = RATE_OF_RECOVERY
        self.minimum_score = MINIMUM_SCORE

    def forward(self, max_score, score, verification_result, factor):
        """
        This method calculates the reward for a miner based on the provided score, verification_result, and factor using a neural network module.
        Positional Arguments:
            max_score (Tensor): The maximum score for the miner.
            score (Tensor): The current score for the miner.
            verification_result (Tensor): Whether the response that the miner submitted was valid. (1 or 0)
            factor (Tensor): The factor to apply to the reward, in case the miner is using multiple hotkeys or serving from the same IP multiple times.
        Returns:
            Tensor: The new score for the miner.
        """
        # Use 'verification_result' to switch between recovery and decay rates
        rate = (
            self.rate_of_decay * (1 - verification_result)
            + self.rate_of_recovery * verification_result
        )

        # Calculate distance based on 'verification_result'
        distance = (max_score - score) * verification_result + (
            score - self.minimum_score
        ) * (1 - verification_result)

        # Apply factor
        new_score = score + rate * distance * factor

        return new_score

In [8]:
"""
Convert the reward method into a zk circuit by exporting to ONNX and then passing through EZKL

"""

import json
import os

# Importing necessary libraries
import bittensor as bt
import ezkl
import numpy as np
import onnxruntime as ort
import torch


if os.getcwd().split("/")[-1] != "model":
    os.chdir("./model")


# Instantiate the Reward model and set it to evaluation mode
reward_model = Reward()
reward_model.eval()
bt.logging.trace(
    f"Instantiated Reward model: {reward_model} and set it to evaluation mode."
)

# Define dummy inputs for the model export
dummy_inputs = {
    "max_score": torch.tensor([1.0], dtype=torch.float32),
    "score": torch.tensor([0.5], dtype=torch.float32),
    "verification_result": torch.tensor([1.0], dtype=torch.float32),
    "factor": torch.tensor([1.0], dtype=torch.float32),
}
bt.logging.trace(f"Defined dummy inputs for the model export: {dummy_inputs}.")

bt.logging.info("Creating zk circuit for the reward function")
# Exporting the model to ONNX format
bt.logging.trace("Starting the export of the model to ONNX format.")
torch.onnx.export(
    reward_model,
    tuple(dummy_inputs.values()),
    "network.onnx",
    opset_version=11,
    input_names=list(dummy_inputs.keys()),
    output_names=["output"],
)
bt.logging.trace("Completed exporting the model to ONNX format: network.onnx.")
# Running inference with ONNX Runtime
bt.logging.trace("Initiating inference with ONNX Runtime on network.onnx.")
ort_session = ort.InferenceSession("network.onnx")
np_inputs = {k: np.array(v, dtype=np.float32) for k, v in dummy_inputs.items()}
output = ort_session.run(
    output_names=["output"],
    input_feed=np_inputs,
)
bt.logging.trace(f"Completed inference with ONNX Runtime. Output: {output}.")
# Saving input and output data to JSON file
bt.logging.trace("Saving the input and output data to a JSON file: input.json.")
with open("input.json", "w", encoding="utf8") as input_file:
    input_json_object = {
        "input_data": [v.tolist() for v in dummy_inputs.values()],
        "output_data": [output[0].tolist()],
    }
    json_str = json.dumps(input_json_object, indent=4)
    input_file.write(json_str)
bt.logging.trace(
    "Saved the input and output data to a JSON file successfully: input.json."
)
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed"
ezkl.gen_settings(py_run_args=py_run_args)
ezkl.calibrate_settings("input.json", target="accuracy")
ezkl.compile_circuit()
ezkl.get_srs()
ezkl.setup(
    model="model.compiled", vk_path="vk.key", pk_path="pk.key", srs_path="kzg.srs"
)
ezkl.gen_witness(
    data="input.json",
    model="model.compiled",
    output="witness.json",
    vk_path="vk.key",
    srs_path="kzg.srs",
)
ezkl.prove(witness="witness.json", model="model.compiled")
ezkl.verify()
bt.logging.success("Successfully circuitized the reward function.")

2024-04-19 01:57:39.930 |       INFO       | Creating zk circuit for the reward function


calibration failed max lookup input (0, 1760936591360) is too large
calibration failed max lookup input (0, 14070312861696) is too large
calibration failed max lookup input (0, 112562502893568) is too large
calibration failed max lookup input (0, 28140625723392) is too large
calibration failed max lookup input (0, 225125005787136) is too large
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
calibration failed max lookup input (0, 450250011574272) is too large


 <------------- Numerical Fidelity Report (input_scale: 13, param_scale: 13, scale_input_multiplier: 10) ------------->

+----------------+----------------+----------------+----------------+----------------+------------------+----------------+----------------+---------------------+--------------------+------------------------+
| mean_error     | median_error   | max_error      | min_error      | mean_abs_error | median_abs_error | max_abs_error  | min_abs

2024-04-19 01:57:40.234 |     SUCCESS      | Successfully circuitized the reward function.


In [47]:
"""
Conduct example proven reward calculations
"""
import copy
# Setup example input
input = [[1.0], [0.7], [1.0], [1.0]]

# Copy into the input.json file
with open("input.json", "w", encoding="utf8") as input_file:
    input_json_object = {
        "input_data": input,
    }
    json_str = json.dumps(input_json_object, indent=4)
    input_file.write(json_str)

# Generate a witness.json file
ezkl.gen_witness(data="input.json", model="model.compiled", output="witness.json")
# Generate a proof
ezkl.prove(witness="witness.json", model="model.compiled", proof_path="proof.json")

with open("proof.json", "r", encoding="utf8") as proof_file:
    proof_data = json.load(proof_file)


print("Resulting score in hex (decimal): ", proof_data["instances"][0][-1],  "(" + proof_data["pretty_public_inputs"]["rescaled_outputs"][0][0] + ")")

print("Attempting to verify with good instances and proof")

# Verify

try:
    ezkl.verify()
    print("Verified")
except:
    print("Not verified")

# Modify

print("Attempting to verify modified instances")

with open("proof.json", "w", encoding="utf8") as proof_file:
    # Modify one of the instances
    modified_instances_proof_data = copy.deepcopy(proof_data)
    modified_instances_proof_data["instances"][0][-1] = modified_instances_proof_data["instances"][0][-1].replace("5", "6")
    json_str = json.dumps(modified_instances_proof_data, indent=4)
    proof_file.write(json_str)

try:
    ezkl.verify()
    print("Verified")
except:
    print("Not Verified")

print("Attempting to verify a modified proof")

with open("proof.json", "w", encoding="utf8") as proof_file:
    # Modify one of the instances
    modified_proof_data = copy.deepcopy(proof_data)
    modified_proof_data["proof"][0] = modified_proof_data["proof"][1]
    json_str = json.dumps(modified_proof_data, indent=4)
    proof_file.write(json_str)

try:
    ezkl.verify()
    print("Verified")
except:
    print("Not Verified")

Resulting score in hex (decimal):  0000000000ae1785010000000000000000000000000000000000000000000000 (0.7599462866783142)
Attempting to verify with good instances and proof
Verified
Attempting to verify modified instances
Not Verified
Attempting to verify a modified proof
Not Verified
